In [13]:
%load_ext autoreload
%load_ext lab_black

%autoreload 2

import os
import sys
sys.path.insert(1, "backend")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_context("paper")

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.backend as K

from tf_backend.tf_models import BigConvNet, OneLayerNet
from tf_backend.tf_losses import pinball_loss_with_scores_keras, dependent_label_quantile_loss_keras, pinball_loss_keras
from tf_backend.tf_utils import *
from tf_backend.tf_metrics import *
from tf_backend.tf_constraints import *

from np_backend.numpy_utils import *
import np_backend.conformal_utils as cf_utils

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds

print("TensorFlow version: ", tf.__version__)
print("TensorFlow Datasets version: ", tfds.__version__)


TensorFlow version:  2.14.0
TensorFlow Datasets version:  4.9.3


# ImageNet

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5088314075429564584
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6309331024377128616
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9904958725853005179
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
data_folder = "datasets/ImageNet/"

dataset_version = "testV2/imagenetv2-matched-frequency/"

path_to_scratch = os.getcwd()
path_to_data = os.path.join(path_to_scratch, data_folder)

n_classes = 1000

IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 32

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy(
        )  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def process_class_name(file_path):
    class_name = tf.strings.split(file_path, '/')[-2]
    return tf.strings.to_number(class_name, tf.int64)


def process_file(file_path):
    image = tf.io.read_file(file_path)
    label = process_class_name(file_path)
    return image, label


def serialize_example(img, label):
    """
      Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {
        'img': _bytes_feature(img),
        'label': _int64_feature(label),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature)
    )

    return example_proto.SerializeToString()


def tf_serialize_example(img, label):
    tf_string = tf.py_function(serialize_example, (img, label), tf.string)
    return tf.reshape(tf_string, ())


def parse_example(img, label):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH]), label


# Create a description of the features.
feature_description = {
    'img': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}


def _parse_function(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    # resize the image to the desired size.
    return parse_example(example["img"], example["label"])


def one_hot_preprocessing(img, label):
    return img, tf.one_hot(tf.reshape(label, [-1]), n_classes, axis=-1)

# Save to tfrecord format (No Need to Run this part)

In [ ]:
# testv2_imagenet_files = tf.data.Dataset.list_files(
#     path_to_data + dataset_version + "*/*.*",
# )

# dataset_len = 0
# for x in testv2_imagenet_files:
#     dataset_len += 1
# print("Number of images: {}".format(dataset_len))

# testv2_imagenet_files = testv2_imagenet_files.map(process_file)

In [ ]:
# for img, label in testv2_imagenet_files.take(1):
#     img, label = parse_example(img, label)
#     print(label)
#     plt.imshow(img.numpy())
#     plt.show()

In [ ]:
# testv2_imagenet_files = testv2_imagenet_files.map(tf_serialize_example)
# testv2_imagenet_files = testv2_imagenet_files.batch(128)

In [ ]:
# tf_writer = tf.io.TFRecordWriter(
#     path_to_data + dataset_version + "data.tfrecord"
# )

# for i, x in enumerate(testv2_imagenet_files):
#     if i % 10 == 0:
#         print(i)
#     for k in range(x.shape[0]):
#         tf_writer.write(x[k].numpy())

# tf_writer.close()

# Load ImageNet Std

In [ ]:
def _processing_imagenet_(example):
    img = tf.image.convert_image_dtype(example["image"], tf.float32)
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH]), example["label"]

In [ ]:
imagenet_data = tfds.load(
    "imagenet2012",
    data_dir=path_to_data "/tensorflow_datasets",
    download=False
)

train_data = imagenet_data["train"]
val_data = imagenet_data["validation"]

In [ ]:
imagenet_data["train"]

In [ ]:
train_data = train_data.map(_processing_imagenet_)
train_data = train_data.batch(batch_size=BATCH_SIZE)
train_data = train_data.map(one_hot_preprocessing)

In [ ]:
val_data = val_data.map(_processing_imagenet_)
val_data = val_data.batch(batch_size=BATCH_SIZE)
val_data = val_data.map(one_hot_preprocessing)

In [ ]:
train_data, val_data

In [ ]:
for x, y in imagenet_data["train"].map(_processing_imagenet_).take(2):
    plt.figure(figsize=(5, 3))
    plt.imshow(x.numpy())
    plt.show()

# Load ImageNet Test V2 Set

Need to download imagenet v2 to a tfrecord dataset before here: https://www.tensorflow.org/datasets/catalog/imagenet_v2

In [ ]:
filename_testv2_tf_record = path_to_data + dataset_version + "data.tfrecord"
tfrecord_testv2_dataset = tf.data.TFRecordDataset(filename_testv2_tf_record)

In [ ]:
testv2_data = tfrecord_testv2_dataset.map(
    _parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE
)

In [ ]:
for x, y in testv2_data.take(2):
    print(y)
    plt.figure(figsize=(5, 3))
    plt.imshow(x.numpy())
    plt.show()

In [ ]:
testv2_data

In [ ]:
testv2_data = testv2_data.batch(batch_size=BATCH_SIZE)
testv2_data = testv2_data.map(one_hot_preprocessing)

In [ ]:
for x, y in testv2_data.take(1):
    print(x.shape, y.shape)

# Load Prediction Model -- Already Trained

In [ ]:
model = BigConvNet(
    name="InceptionResNetV2",
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    output_shape=1000,
    include_top=True
)

Pre-Trained Predictive Network (achieving about 93% accuracy)

In [ ]:
# model_weights_path = path_to_data + "/models" + "/weights_ResNet50V2CIFAR_pretrained.h5"
# model.load_weights(model_weights_path)

optimizer_args = {"learning_rate": 0.1, "momentum": 0.9, "nesterov": True}
optimizer = optimizer_dict["SGD"](**optimizer_args)
metrics_list = [tf.keras.metrics.CategoricalAccuracy(name="accuracy")]

model.compile(
    optimizer,
    loss="categorical_crossentropy",
    metrics=metrics_list,
)

In [ ]:
# model.summary()

# Check model performance

In [ ]:
model.evaluate(train_data)

In [ ]:
model.evaluate(val_data)

In [ ]:
model.evaluate(testv2_data)

# Save Features to numpy

In [ ]:
n_features = 1536
model_preprocessing = tf.keras.Model(
    inputs=model.input, outputs=model.layers[-2].output
)

In [ ]:
dataset_split_list = ["val", "test"]

In [ ]:
imagenet_datasets = {"val": val_data, "test": testv2_data}

In [ ]:
scores = {}
labels = {}
features = {}

for dataset in imagenet_datasets:
    features[dataset] = np.empty((0, n_features))
    labels[dataset] = np.empty((0, n_classes))
    scores[dataset] = np.empty((0, n_classes))
    for x, y in imagenet_datasets[dataset]:
        features[dataset] = np.concatenate(
            (features[dataset], model_preprocessing(x).numpy()), axis=0
        )
        scores[dataset] = np.concatenate(
            (scores[dataset], np.log(np.maximum(1e-100,
                                                model(x).numpy()))),
            axis=0
        )
        labels[dataset] = np.concatenate((labels[dataset], y.numpy()), axis=0)

In [ ]:
for dataset in imagenet_datasets:
    scores[dataset] = np.maximum(-100.0, scores[dataset])

In [ ]:
for dataset in dataset_split_list:
    np.save(
        path_to_data + "np_data/" + dataset + ".npy", {
            "features": features[dataset],
            "labels": labels[dataset],
            "scores": scores[dataset]
        }
    )

You can load train, val, and testV2 data already preprocessed directly in numpy format, at the following location

In [10]:
f"{path_to_data}np_data"

'/Users/maxcauch/Documents/Stanford/phd-research/robust_cv/code/reproducible_code_for_jasa_alnur/datasets/ImageNet/np_data'

In [ ]:
np.load(path_to_data + "np_data/" + dataset + ".npy",
        allow_pickle=True).item()["features"].shape